In [1]:
import sys
import json
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback
sys.path.append("../../../")
from src.Evaluation import plot_model_history
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder
from src.LoadData import get_all_datasets_test_train_np_arrays
from src.Helpers import append_to_csv

2023-01-18 18:04:33.457751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 18:04:33.588527: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-18 18:04:34.456175: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-18 18:04:34.456246: W tensorflow/compil

In [2]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8


In [3]:
path_to_datasets = "../../../datasets"

In [4]:
datasets_test_train_data = get_all_datasets_test_train_np_arrays(path_to_datasets)

In [5]:
models_getter = [get_Encoder, ]
models_names = ["Encoder", ]

In [6]:
# tf.debugging.set_log_device_placement(True)
devices = tf.config.list_logical_devices('GPU') # Use this to run training just on GPUs
#devices = tf.config.list_logical_devices()
strategy = tf.distribute.MirroredStrategy(devices)

2023-01-18 18:04:43.990256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 18:04:45.140098: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30952 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2023-01-18 18:04:45.140998: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30952 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [7]:
devices

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [8]:
path_persist_results = "./training_res.csv"
if not (csv_path := Path(path_persist_results)).exists():
    csv_path.touch()
if len(csv_path.read_text()) == 0:
    # Only write if the file is empty
    columns = ["dataset_name", "model_name", "test_loss", "test_acc", "confusion_matrix", "history"]
    append_to_csv(path_persist_results, columns)

In [9]:
epochs = 30
batch_size = 10
validation_split = 0.2

In [10]:
with strategy.scope():
    for ds_name, ds_data in tqdm(datasets_test_train_data.items(), unit='dataset'):
        print("Dataset name: ", ds_name)
        x_test, y_test = ds_data["test_data"]
        x_train, y_train = ds_data["train_data"]

        input_size = x_train.shape[1]
        output_size = len(np.unique(y_train))

        for get_model, model_name in tqdm(list(zip(models_getter, models_names)), unit='model', desc=f'Train on "{ds_name}"'):
            print("Model name: ", model_name)
            model = get_model(input_size, output_size)
            model.compile(optimizer='SGD',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])
            history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[TqdmCallback(verbose=0, desc=model_name)], verbose=0)
            test_loss, test_acc = model.evaluate(x_test, y_test)
            # TODO: add confusion matrix

            row = [ds_name,
                  model_name,
                  test_loss,
                  test_acc,
                  None, # TODO: persist confusion matrix to further analysis
                  json.dumps(history.history)]
            append_to_csv(path_persist_results, row)

  0%|          | 0/11 [00:00<?, ?dataset/s]

Dataset name:  abnormal_heartbeat


Train on "abnormal_heartbeat":   0%|          | 0/1 [00:00<?, ?model/s]

Model name:  Encoder
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Encoder: 0epoch [00:00, ?epoch/s]

...

6/6 [==============================] - 0s 9ms/step - loss: 0.6915 - accuracy: 0.6400
Dataset name:  wafer


Train on "wafer":   0%|          | 0/1 [00:00<?, ?model/s]

Model name:  Encoder


Encoder: 0epoch [00:00, ?epoch/s]

193/193 [==============================] - 3s 5ms/step - loss: 0.0222 - accuracy: 0.9933


# Example load training results and display evaluation results

In [11]:
df_training_res = pd.read_csv(path_persist_results)

In [12]:
df_training_res

,dataset_name,model_name,test_loss,test_acc,confusion_matrix,history
0,abnormal_heartbeat,Encoder,1.260185,0.726829,NaN,"{""loss"": [0.5163329243659973, 0.05385819450020..."
1,phalanges_outlines_correct,Encoder,0.508525,0.755245,NaN,"{""loss"": [0.6279335021972656, 0.61735367774963..."
2,swedish_leaf,Encoder,0.286636,0.915200,NaN,"{""loss"": [2.610792875289917, 2.172828912734985..."
3,share_price_increase,Encoder,1.290622,0.687371,NaN,"{""loss"": [0.4948023557662964, 0.40449222922325..."
4,distal_phalanx_outline,Encoder,0.550073,0.735507,NaN,"{""loss"": [0.6747336983680725, 0.62875455617904..."
5,electric_devices,Encoder,1.470596,0.691739,NaN,"{""loss"": [1.189337134361267, 0.798384010791778..."
6,freezers,Encoder,0.133145,0.940702,NaN,"{""loss"": [0.6313468813896179, 0.61874383687973..."
7,cricket_y,Encoder,1.721979,0.502564,NaN,"{""loss"": [2.460876226425171, 2.235610485076904..."
8,coffee_beans,Encoder,0.485757,0.642857,NaN,"{""loss"": [0.7004410028457642, 0.66082811355590..."
9,arrow_head,Encoder,0.691487,0.640000,NaN,"{""loss"": [1.1015182733535767, 1.06762242317199..."


In [14]:
history = df_training_res['history'][0]

In [15]:
json.loads(history)

{'loss': [0.5163329243659973,
  0.0538581945002079,
  0.00482696108520031,
  0.001751854782924056,
  0.0016706426395103335,
  0.0007307759369723499,
  0.0006830706843174994,
  0.0004944929969497025,
  0.0004373743140604347,
  0.0009648362174630165,
  0.0005137254483997822,
  0.00031668427982367575,
  0.00028367075719870627,
  0.00027524339384399354,
  0.00031098644831217825,
  0.00021740842203143984,
  0.00024244197993539274,
  0.00027552287792786956,
  0.00020925352873746306,
  0.0001911270956043154,
  0.0002227224176749587,
  0.00016026344383135438,
  0.00018987749353982508,
  0.00026405006065033376,
  0.00015174208965618163,
  9.252057498088107e-05,
  0.0001497379707871005,
  0.0001423564535798505,
  0.0001247290929313749,
  0.0001800397876650095],
 'accuracy': [0.7361963391304016,
  0.9815950989723206,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,


In [ ]:
plot_model_history(json.loads(history), epochs=epochs)